## Coroutines

In [1]:
# Simplest coroutine
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)

my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x109a26b88>

In [2]:
next(my_coro)

-> coroutine started


In [3]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [4]:
my_coro = simple_coroutine()
my_coro.send(1729)

TypeError: can't send non-None value to a just-started generator

In [7]:
my_coro.send(None)

-> coroutine started


In [8]:
my_coro.send(None)

-> coroutine received: None


StopIteration: 

In [10]:
my_coro.send(42)

StopIteration: 

In [11]:
# A coroutine that yields twice
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

my_coro2 = simple_coro2(14)

In [12]:
from inspect import getgeneratorstate
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [13]:
next(my_coro2)

-> Started: a = 14


14

In [14]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [15]:
my_coro2.send(28)

-> Received: b = 28


42

In [16]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [17]:
# To compute a running average
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

coro_avg = averager()
next(coro_avg)
coro_avg.send(10)

10.0

In [18]:
coro_avg.send(30)

20.0

In [19]:
coro_avg.send(5)

15.0

In [20]:
coro_avg.close()

In [ ]:
# decorator for priming coroutine
# coroutil.py
from functools import wraps

def coroutine(func):
    """Decorator: primes 'func' by advancing to first 'yield'"""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [31]:
from coroutil import coroutine

@coroutine
def averager1():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

coro_avg1 = averager1()
coro_avg1.send(10)

10.0

In [32]:
coro_avg1.send(30)

20.0

In [34]:
coro_avg1.send(5)

15.0

In [35]:
coro_avg1.send('bad')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [38]:
# test code for studying exception handling in a coroutine
# coro_exc_demo.py
class DemoException(Exception):
    """An exception type for the demonstration"""
    
def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')